# Product Owner LoRA Fine-tuning en Google Colab

**Objetivo**: Entrenar modelo Product Owner usando Qwen2.5-7B + LoRA

**Requirements**: Google Colab Free con GPU T4 (15GB VRAM)

**Tiempo estimado**: 3-5 horas para 3 epochs con ~200 samples

**Setup**: Solo necesitas clonar el repo (el dataset está incluido)

---

## 1️⃣ Verificar GPU y CUDA

In [ ]:
# Verificar GPU disponible
!nvidia-smi

# Verificar PyTorch y CUDA
import torch
print(f"\n{'='*60}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"\n✅ GPU is ready!")
else:
    print(f"\n❌ No GPU detected!")
    print(f"   Runtime → Change runtime type → Hardware accelerator: T4 GPU")
print(f"{'='*60}")

## 2️⃣ Instalar Dependencias

In [ ]:
# Instalar todas las librerías necesarias
print("📦 Installing dependencies...\n")

!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers==4.36.0 peft==0.7.0 bitsandbytes==0.41.0 accelerate==0.25.0 datasets==2.16.0 typer

# Verificar instalación
print("\n🔍 Verifying installation...\n")
import transformers
import peft
import bitsandbytes
import datasets
import typer

print(f"✅ transformers: {transformers.__version__}")
print(f"✅ peft: {peft.__version__}")
print(f"✅ bitsandbytes: {bitsandbytes.__version__}")
print(f"✅ datasets: {datasets.__version__}")
print(f"\n✅ All packages installed successfully!")

## 3️⃣ Clonar Repositorio desde GitHub

In [ ]:
# Clonar el repositorio desde el branch correcto (incluye el dataset)
print("📥 Clonando repositorio desde GitHub...\n")

!git clone -b dspy-multi-role https://github.com/krukmat/agnostic-ai-pipeline.git
%cd agnostic-ai-pipeline

# Verificar branch y dataset
print("\n🔍 Verificando branch y dataset...\n")
!git branch --show-current
!ls -lh artifacts/distillation/po_teacher_supervised.jsonl

print("\n✅ Repositorio clonado exitosamente en branch dspy-multi-role!")

## 4️⃣ Verificar Dataset y Script

In [ ]:
import json
import os
from pathlib import Path

print("🔍 Verificando archivos necesarios...\n")

# Paths
script_path = Path("/content/agnostic-ai-pipeline/scripts/train_po_lora.py")
dataset_path = Path("/content/agnostic-ai-pipeline/artifacts/distillation/po_teacher_supervised.jsonl")

# Verificar script
if script_path.exists():
    print(f"✅ Script encontrado: {script_path}")
else:
    print(f"❌ Script NO encontrado: {script_path}")

# Verificar dataset
if dataset_path.exists():
    print(f"✅ Dataset encontrado: {dataset_path}")
    
    # Contar registros
    with open(dataset_path) as f:
        samples = [line for line in f if line.strip()]
        print(f"✅ Dataset tiene {len(samples)} registros")
    
    # Ver primer sample
    with open(dataset_path) as f:
        first = json.loads(f.readline())
        print(f"\n📝 Keys: {list(first.keys())}")
        print(f"📏 Prompt length: {len(first['prompt'])} chars")
        print(f"📏 Response length: {len(first['response'])} chars")
        print(f"\n🔍 Preview prompt:")
        print(first['prompt'][:300] + "...")
        print(f"\n✅ Dataset format is correct!")
else:
    print(f"❌ Dataset NO encontrado: {dataset_path}")
    print(f"\n🔍 Buscando dataset en el repo...")
    !find /content/agnostic-ai-pipeline -name "po_teacher_supervised.jsonl" 2>/dev/null

## 5️⃣ Ejecutar Training

**Parámetros optimizados para T4 (15GB VRAM)**:
- 4-bit quantization para reducir memoria
- Gradient checkpointing activado
- Batch size 1 + gradient accumulation 8 = batch efectivo de 8
- Max length 2048 (ajusta a 1536 si hay OOM)

**Tiempo estimado**: 3-5 horas

In [ ]:
# Ejecutar training
!python scripts/train_po_lora.py \
    --data-path /content/agnostic-ai-pipeline/artifacts/distillation/po_teacher_supervised.jsonl \
    --base-model Qwen/Qwen2.5-7B-Instruct \
    --output-dir /content/agnostic-ai-pipeline/artifacts/models/po_student_v1 \
    --rank 32 \
    --alpha 64 \
    --dropout 0.05 \
    --epochs 3 \
    --batch-size 1 \
    --gradient-accumulation-steps 8 \
    --lr 1e-4 \
    --max-length 2048 \
    --load-4bit \
    --bnb-compute-dtype float16 \
    --gradient-checkpointing

### 🚨 Si el training falla con "CUDA out of memory", ejecuta esta celda alternativa:

In [ ]:
# Comando ALTERNATIVO con parámetros más conservadores
# Descomenta y ejecuta si el comando anterior falló con OOM

# !python scripts/train_po_lora.py \
#     --data-path /content/agnostic-ai-pipeline/artifacts/distillation/po_teacher_supervised.jsonl \
#     --base-model Qwen/Qwen2.5-7B-Instruct \
#     --output-dir /content/agnostic-ai-pipeline/artifacts/models/po_student_v1 \
#     --rank 16 \
#     --alpha 32 \
#     --dropout 0.05 \
#     --epochs 3 \
#     --batch-size 1 \
#     --gradient-accumulation-steps 16 \
#     --lr 1e-4 \
#     --max-length 1536 \
#     --load-4bit \
#     --bnb-compute-dtype float16 \
#     --gradient-checkpointing

## 6️⃣ Monitorear GPU (Ejecutar en otra celda mientras entrena)

In [ ]:
# Ver uso de GPU en tiempo real (actualiza cada 5 segundos)
!nvidia-smi --query-gpu=utilization.gpu,utilization.memory,memory.used,memory.total --format=csv -l 5

## 7️⃣ Verificar Checkpoints Guardados

In [ ]:
# Ver checkpoints guardados después del training
print("📁 Checkpoints guardados:\n")
!ls -lh /content/agnostic-ai-pipeline/artifacts/models/po_student_v1/

## 8️⃣ Test de Inferencia Rápido

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

print("🔄 Loading model...\n")

base_model = "Qwen/Qwen2.5-7B-Instruct"
adapter_path = "/content/agnostic-ai-pipeline/artifacts/models/po_student_v1"

# Cargar tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Cargar modelo base en 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)

# Cargar adapter LoRA
model = PeftModel.from_pretrained(model, adapter_path)

print("✅ Model loaded!\n")

# Test prompt
test_prompt = """[INSTRUCTIONS]
You are a Product Owner AI agent validating business requirements.

[REQUIREMENTS]
business_domain: Blog Platform
primary_features:
  - Users can create blog posts with title and content
  - Posts must support markdown formatting
  - User authentication is required

[YOUR RESPONSE]"""

print("💭 Test prompt:")
print(test_prompt)
print("\n" + "="*60 + "\n")

# Generate
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("🤖 Model response:")
print(response[len(test_prompt):])

## 9️⃣ Descargar Modelo Entrenado

**Opción recomendada**: Comprimir y descargar

In [ ]:
from google.colab import files

print("📦 Compressing model...\n")
!zip -r po_student_v1.zip /content/agnostic-ai-pipeline/artifacts/models/po_student_v1

print("\n⬇️ Downloading...")
files.download('po_student_v1.zip')

print("\n✅ Download complete!")

---

## 🔧 Troubleshooting

### Error: CUDA out of memory
**Solución**: Ejecuta la celda alternativa (Cell 5.2) con parámetros más conservadores

### Error: No module named 'bitsandbytes'
**Solución**: Reinstala bitsandbytes
```python
!pip uninstall -y bitsandbytes
!pip install bitsandbytes==0.41.0 --no-cache-dir
```

### Error: No GPU detected
**Solución**: 
1. Runtime → Change runtime type
2. Hardware accelerator: **T4 GPU**
3. Save y reconectar

### Training muy lento (< 1 it/s)
**Solución**: Usa la celda alternativa con `--max-length 1536` o `--gradient-accumulation-steps 4`

---

## 📊 Métricas Esperadas

```
Epoch 1/3: 100%|██████████| 25/25 [12:34<00:00,  0.05it/s]
{'loss': 1.2345, 'learning_rate': 9.5e-05, 'epoch': 1.0}

Epoch 2/3: 100%|██████████| 25/25 [12:31<00:00,  0.05it/s]
{'loss': 0.8912, 'learning_rate': 5.0e-05, 'epoch': 2.0}

Epoch 3/3: 100%|██████████| 25/25 [12:29<00:00,  0.05it/s]
{'loss': 0.6234, 'learning_rate': 5.0e-06, 'epoch': 3.0}
```

**Indicadores de éxito**:
- ✅ Loss decreciente (1.2 → 0.6)
- ✅ No crashes por OOM
- ✅ ~0.03-0.05 it/s en T4
- ✅ Checkpoints guardados cada epoch

---

## 📚 Referencias

- **Setup Guide**: `docs/COLAB_TRAINING_SETUP.md`
- **Training Script**: `scripts/train_po_lora.py`
- **HuggingFace Qwen2.5**: https://huggingface.co/Qwen/Qwen2.5-7B-Instruct
- **PEFT Docs**: https://huggingface.co/docs/peft/

---

**Última actualización**: 2025-11-13